# 20180930 Update

- reorg codes
	- flex
	- clearness
	- we should memic the strucutr of DiDi's OD flow prediction model code:
		- one function -> build model (different model strucutre)
		- one function -> train the model (specify small/large UCF or other dataset)
		- oen function -> inquiry on UCF
		- one function get accuracy description and save to txt
- run experiments
	- DTW vs. simple matching (need 2 differnt length of video in inquiry set)
	- ConvLSTM vs. LSTM vs. Conv2D (sort of frame based VGG ) vs. (VidSig) (Non-DL method)

# A simple test for inquiry model


Here is the setting:

- My thought: Yintai Ma [09:23] 
  - Suppose I have a video X of 100 frames, denoting as X[0] to X[99]. We then split it to multiple clips without overlap. If then length of each clip is 5 frames, then we come up with a sequences of clips X[0:5],X[5:10],...,X[95:100]. We use the encoder to transfer the sequences of clips to sequences of embeddeds, reads y[0],y[1],...,y[20]. Now we randomly pick some frames from X as validation clip and transfer it into validation embedded y_hat. We want to see how y_hat is matching to the sequences of y[0]...y[20].

- Diego:
  - You have video X. Randomly pick  sequences of frames (non overlapping). Say x[4:10], x[34:37],x[85:95]. Now concatenate them into a single video. This is now your query. From here, create embedding sequence y[0],…y[K]. Now do DTW of y against your encoded sequences of the videos in the database.
  - This is subject to experimentation. I agree that there should be overlap. Overlap by half.

- My thought: There are many variations for the implementation:
  1. we keep overlaps for y. We transfer X[0:5],X[1:6],....X[94:99],X[95:100] into y[0]...y[100]. Now we want to see how y_hat is matching to y[].
  2. When we pick some frames from X to composite validation clips, do we always pick 5 consecutive frames? Should we ever transfer X[0]+X[2:6] into y_hat?
  3. How we match y_hat to y[] if y_hat is also a sequences? I think this is where DTW comes in right? If y_hat is just one embedded, then what we need is basically a simple comparison between the distance of two embedded. However, if y_hat is a sequences, say it has y_hat[0] and y_hat[1], then we will need to use DTW to consider the case where both y_hat[0] matches to y[0] and y_hat[1] matches to y[3] are the best query retrieve.


In [1]:
import sys,os,os.path
sys.path.append(os.path.expanduser('/home/lab.analytics.northwestern.edu/yma/git/videodl/seq_inquiry'))
os.environ['CUDA_VISIBLE_DEVICES']='3'

from keras.models import load_model

import csv
import os
import time
import sys
import numpy as np
import matplotlib
# matplotlib.use('Agg')
import pylab as plt
# from scipy.misc import toimage
import pickle

import imageio
import cv2
import numpy as np
import numpngw
import pandas as pd 

from IPython.display import HTML
import random

from mymodels import *
from Video2videoInquiry import *

import data_seq

/home/lab.analytics.northwestern.edu/yma/.conda/envs/dl/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/lab.analytics.northwestern.edu/yma/.conda/envs/dl/lib/python2.7/site-packages/scipy/optimize/_minimize.py:32: ImportWarning: Not importing directory '/home/lab.analytics.northwestern.edu/yma/.conda/envs/dl/lib/python2.7/site-packages/scipy/optimize/lbfgsb': missing __init__.py
  from .lbfgsb import _minimize_lbfgsb
/home/lab.analytics.northwestern.edu/yma/.conda/envs/dl/lib/python2.7/site-packages/scipy/spatial/__init__.py:95: ImportWarning: Not importing directory '/home/lab.analytics.northwestern.edu/yma/.conda/envs/dl/lib/python2.7/site-packages/scipy/spatial/qhull': missing __init__.py
  from .qhull import *

# Load trained model

In [24]:
LOAD_DATABASE_FROM_PKL = True

# import conv_ae_config as config
# model_file = "ucf_vgg16_seq3_convlstm.001-0.0689.hdf5" # encoding filter = 8
# encoder, autoencoder =  ConvAutoEncoder(sequenceLength = config.sequenceLength)
# database_file = "/scratch/yma/data/inq_encoded_class100_video500_conv_convlstm_8.pkl"

import simple_convlstm_ae_config as config
setup = "simple_convlstm"
model_file = "ucf_seq3_simple_convlstm.003-0.0284.hdf5" # encoding filter = 8
encoder, autoencoder =  SimpleConvLstmAutoEncoder(sequenceLength = config.sequenceLength)
# database_file = "/scratch/yma/data/inq_encoded_class100_video500_simple_convlstm_8.pkl"
database_file = "/scratch/yma/data/inq_encoded_class100_video5000_simple_convlstm_8.pkl"

# import simple_conv_ae_config as config
# setup = "simple_conv"
# model_file = "ucf_seq3_simple_conv.014-0.0114.hdf5" # encoding filter = 8
# encoder, autoencoder =  SimpleConvAutoEncoder(sequenceLength = config.sequenceLength)
# database_file = "/scratch/yma/data/inq_encoded_class100_video500_simple_conv_8.pkl"
# database_file = "/scratch/yma/data/inq_encoded_class100_video5000_simple_conv_8.pkl"

# import simple_lstm_ae_config as config
# setup = "simple_lstm"
# model_file = "ucf_seq3_simple_lstm.002-0.0726.hdf5" # encoding filter = 8
# encoder, autoencoder =  SimpleLstmAutoEncoder(sequenceLength = config.sequenceLength)
# encoder, autoencoder =  SimpleLstmAutoEncoder(sequenceLength = config.sequenceLength)
# database_file = "/scratch/yma/data/inq_encoded_class100_video500_simple_lstm_400.pkl"
# database_file = "/scratch/yma/data/inq_encoded_class100_video5000_simple_lstm_400.pkl"

model_dir = "/home/lab.analytics.northwestern.edu/yma/git/data/checkpoints/"
data_path = "/scratch/yma/git/five-video-classification-methods/data"

encoder.load_weights(os.path.join(model_dir, model_file), by_name=True)
print("The output dimension of encoder is:", encoder.output.shape)

# for model_dict in models:
#     print("Loading weights for setup:", model_dict["setup"])
#     model_file = os.path.join(model_dir, model_dict["model_file"])
#     model_dict["encoder"].load_weights(model_file, by_name=True)
#     print("The output dimension of encoder is:", model_dict["encoder"].output.shape)

('Encoded embedding size: ', TensorShape([Dimension(None), Dimension(None), Dimension(7), Dimension(7), Dimension(8)]))
--- Defining Decoder ---
--- Finish Compile and Plot Model ---
('The output dimension of encoder is:', TensorShape([Dimension(None), Dimension(None), Dimension(7), Dimension(7), Dimension(8)]))


In [25]:
SEQ_LENGTH = config.sequenceLength
# the number of frames in each clip

# N_database = 500
# N_database = 5
# N_database = 50

inq_length = 4
# the number of clips in the inquiry, no overlap

DATASET_CLASS_LIMIT = 100
# number of class is the dataset

DATASET_VIDEO_IN_CLASS_LIMIT = 50

FLAG_RANDOM_CLASS = True
# whether randomly pick classes in the dataset


In [26]:
data = initilize(encoder, data_path = data_path, 
                 seq_length = config.sequenceLength, 
                 class_limit = DATASET_CLASS_LIMIT, 
                 num_video_in_each_class = DATASET_VIDEO_IN_CLASS_LIMIT,
                 random_class=FLAG_RANDOM_CLASS)

with open(database_file, 'rb') as f:
    database = pickle.load(f)
print("Database Loaded.")

('Number of records in database:', 11881)
('Number of records in filtered database:', 3383)
Database Loaded.


In [35]:
import pandas as pd
def ComputeTopK(inq_dict, inq_result, KinTopK):
    TopKColumns = []
    for i in range(1, KinTopK+1):
        TopKColumns.append("Top%d"%i+"CH") # ClassHit
    for i in range(1, KinTopK+1):
        TopKColumns.append("Top%d"%i+"CS") # ClassScore
    
    TopKClassHit = []
    for i in range(0, KinTopK):
        if get_category(inq_result["scores"][i][1]) == get_category(inq_dict["smp"][2]):
            TopKClassHit.append(1)
        else:
            TopKClassHit.append(0)
            
    TopKClassScore = []
    for i in range(0, KinTopK):
        TopKClassScore.append(inq_result["scores"][i][0])
        
    return TopKColumns, TopKClassHit, TopKClassScore

def make_pd_df(inq_dict, inq_result, database, DATASET_CLASS_LIMIT, 
               DATASET_VIDEO_IN_CLASS_LIMIT, run_count, KinTopK, Method):
    TopKColumns, TopKClassHit, TopKClassScore = ComputeTopK(inq_dict, inq_result, KinTopK)
    df_columns = ["Setup", 
                  "TestNo",
                 "InqClip",
                  "NClass",
                 "NVideoinClassLimit",
                  "Method",
                 ]+TopKColumns
    print(df_columns)
    df = pd.DataFrame([[setup, 
                        run_count,
                       inq_dict["smp"][2],
                        DATASET_CLASS_LIMIT,
                        DATASET_VIDEO_IN_CLASS_LIMIT,
                        Method,
                       ]+TopKClassHit+TopKClassScore], 
                 columns=df_columns)
    print(df)
    return df

KinTopK = 10
df_dtw = make_pd_df(inq_dict, inq_result, database, DATASET_CLASS_LIMIT, DATASET_VIDEO_IN_CLASS_LIMIT, run_count, KinTopK, Method = "dtw")

['Setup', 'TestNo', 'InqClip', 'NClass', 'NVideoinClassLimit', 'Method', 'Top1CH', 'Top2CH', 'Top3CH', 'Top4CH', 'Top5CH', 'Top6CH', 'Top7CH', 'Top8CH', 'Top9CH', 'Top10CH', 'Top1CS', 'Top2CS', 'Top3CS', 'Top4CS', 'Top5CS', 'Top6CS', 'Top7CS', 'Top8CS', 'Top9CS', 'Top10CS']


AssertionError: 26 columns passed, passed data had 16 columns

In [23]:
# inqs = get_inquiry(data, if_random = True)
# seqY = inqs[1]

KinTopK = 10

run_count = 0

inq_dict, inq_result = inquiry_in_database(encoder, data, database, config, inq_length = inq_length , match_method = "dtw")
df_dtw = make_pd_df(inq_dict, inq_result, database, DATASET_CLASS_LIMIT, DATASET_VIDEO_IN_CLASS_LIMIT, run_count, KinTopK)
show_inquriy_stats(inq_dict, inq_result, show_top_limit = KinTopK)

print("*"*20)
print("*"*20)
print("*"*20)

run_count = 1

inq_dict, inq_result = inquiry_in_database(encoder, data, database, config, 
                                           inq_length = inq_length, match_method = "naive", 
                                           Given_inquiry = True,
                                           inq_dict = inq_dict)
show_inquriy_stats(inq_dict, inq_result, show_top_limit = KinTopK)

NameError: global name 'setup' is not defined

In [20]:
for i in database:
    if "WallPushups" in i[1][2]:
        print(i[1][2])

v_WallPushups_g02_c01
v_WallPushups_g04_c01
v_WallPushups_g05_c04
v_WallPushups_g05_c03
v_WallPushups_g06_c01
v_WallPushups_g07_c01
v_WallPushups_g06_c04
v_WallPushups_g07_c06
v_WallPushups_g07_c04
v_WallPushups_g03_c03
v_WallPushups_g02_c04
v_WallPushups_g07_c02
v_WallPushups_g03_c05
v_WallPushups_g05_c01
v_WallPushups_g04_c02
v_WallPushups_g03_c04
v_WallPushups_g06_c05
v_WallPushups_g03_c02
v_WallPushups_g01_c03
v_WallPushups_g07_c03
v_WallPushups_g07_c05
v_WallPushups_g02_c03
v_WallPushups_g06_c03
v_WallPushups_g01_c02
v_WallPushups_g06_c06
v_WallPushups_g04_c03
v_WallPushups_g02_c02
v_WallPushups_g05_c05
v_WallPushups_g06_c02
v_WallPushups_g01_c01
v_WallPushups_g01_c04
v_WallPushups_g03_c01
v_WallPushups_g06_c07
v_WallPushups_g04_c04
v_WallPushups_g05_c02


In [6]:
# (top_cat_same,top_cat_same_hit, Nth_score_avg, Hit_itself_avg) = multiple_test(data, run_times=100, if_itself=False)

res_dict = multiple_test(models, test_run_for_each_database=5)

NameError: name 'models' is not defined

In [8]:
database[1][1]

['test', 'HorseRiding', 'v_HorseRiding_g04_c02', '201']